# Compare Tsfel and MKR implementation

In [1]:
import numpy as np
import pandas as pd
import swifter
import math
from functools import partial

In [2]:
import inspect
import json

In [3]:
import transformers.features as features

import features as mkr
import tsfel as tsfel

In [4]:
checkValues = True
checkPerf = True
numberComps = 10 ** 3 // 2 # reduce if this takes too much time
# 10**3 takes roughly 2,5h
checkRealWorld = True

Create a random example feature sequence with 15 channels, 2 windows and 50samples in each window. 

The data has shape: (nr windows, nr channels, nr samples)

In [5]:
windows = 2
channels = 15
samples = 50
totalSize = windows * channels * samples

wts_flat = np.random.rand(windows * channels * samples) - 0.5
wts_rand = wts_flat.reshape((windows, channels, samples))

print('Nr Windows: %s, Nr Channels: %s, Nr Values: %s' % (windows, channels, samples))

Nr Windows: 2, Nr Channels: 15, Nr Values: 50


In [6]:
overview = pd.read_csv('mapping.csv')

In [7]:
# all available feature names for adapted features
overview['MKR_ByName'] = overview['MKR'].fillna('tsfel:' + overview['TSFEL']).to_list()

In [8]:
def is_nan(val):
    try:
        float(val)
        return math.isnan(val)
    except:
        return False

def get_params(location, fnname):
    if is_nan(fnname):
        return math.nan
    return list(inspect.signature(getattr(location, fnname)).parameters)[1:]

overview['TSFEL_params'] = overview['TSFEL'].apply(partial(get_params, tsfel))
overview['MKR_params'] = overview['MKR'].apply(partial(get_params, mkr))

In [9]:
def cstr(s):
    # yes, we are using == True here in order to catch NaNs
    return "<text style=color:{}>{}</text>".format('green' if s == True else 'red', s)

def cnum(s):
    return "<text style=color:{}>{}</text>".format('green' if s < 1 else 'red', s)

# Compare Values

Check we did not make an error in the implementation.

1. Calculate values on single channel and compare if equal
2. Calculate values on multi channel and compare if equal

In [10]:
def ref_wrapper(fn, X, **fnArgs):
    res = np.array([fn(ts, **fnArgs) for ts in X.reshape(-1, X.shape[-1])])
    return res.reshape(X.shape[:-1]) if res.size == np.prod(X.shape[:-1]) else res.reshape((*X.shape[:-1], -1))

Default Values for the parameterized features

In [11]:
default_mkr = {
    'samplingfrequency': 1000,
    'prob': 'kde',
    'd': 10
}

default_tsfel = {
    'fs': 1000,
    'prob': 'kde',
    'd': 10
}

In [12]:
def compare_values(wts, row=['calc_mean', 'calc_mean', [], []]):
    mkr_name, tsfel_name, mkr_params, tsfel_params = row
    
    if is_nan(mkr_name) or is_nan(tsfel_name):
        return False
    else:
        fn = getattr(mkr, mkr_name)
        reffn = partial(ref_wrapper, getattr(tsfel, tsfel_name))
    
    try:
        val1 = fn(wts, **{key: default_mkr[key] for key in mkr_params if key in default_mkr})
        val2 = reffn(wts, **{key: default_tsfel[key] for key in tsfel_params if key in default_tsfel})
        np.testing.assert_almost_equal(val1, val2)
        return True
    except Exception as err:
        print('=== %s ===' % mkr_name)
        print(err)
#         raise
        return False

In [13]:
if checkValues:
    # single channel value check
    overview['single_channel'] = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params']].apply(partial(compare_values, wts_rand[0][0]), axis=1)

=== auc ===

Arrays are not almost equal to 7 decimals

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 2.1156802
Max relative difference: 0.99596457
 x: array(0.0085723)
 y: array(2.1242525)


In [14]:
bounds = (2, 15, 50)
tmp = np.arange(2*15*50).reshape(bounds)
tmp1 = []
for x in tmp:
    interm = []
    for y in x:
        interm.append(max(y))
    tmp1.append(interm)
tmp1 = np.array(tmp1)
tmp2 = np.max(tmp.reshape((-1, 50)), axis=-1).reshape(bounds[:-1])
np.testing.assert_almost_equal(tmp1, tmp2)
tmp.shape, tmp1.shape, tmp2.shape

((2, 15, 50), (2, 15), (2, 15))

In [15]:
if checkValues:
    # multi channel value check
    # uses wrapper of style [feature(channel) for channel in multi_channel] for tsfel library
    overview['multi_channel'] = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params']].apply(partial(compare_values, wts_rand), axis=1)

=== auc ===

Arrays are not almost equal to 7 decimals

Mismatched elements: 30 / 30 (100%)
Max absolute difference: 3.01135539
Max relative difference: 1.04148721
 x: array([[0.0085723, 0.0076013, 0.0090313, 0.0083032, 0.0075944, 0.007113 ,
        0.0093734, 0.007575 , 0.0089848, 0.0075855, 0.0077579, 0.0070131,
        0.0088428, 0.0094691, 0.0092957],...
 y: array([[ 2.1242525, -0.7774958,  0.2088851, -0.2910628, -0.2476623,
        -1.2175913, -1.7477649, -0.3246157,  0.4019342,  0.0818663,
        -1.2301714,  0.1546516, -0.7716847, -0.3680342,  3.0206511],...


/app/libraries/src/features/statistical.py:149: RuntimeWarning: divide by zero encountered in log2
  entr = -np.sum(p_without_null * np.log2(p_without_null), axis=-1) / normTerm


Overview of adapted features.

Two left most columns show if the values for single channel signals and multi channel signals match between tsfel and adapted implementation.  
As can be seen several features like mfcc, wavelets have not been adapted.

In [16]:
overview.sort_values(by=['MKR']).style.format({'single_channel': cstr, 'multi_channel': cstr})

,TSFEL,MKR,MKR_ByName,TSFEL_params,MKR_params,single_channel,multi_channel
0,abs_energy,abs_energy,abs_energy,[],[],True,True
1,auc,auc,auc,['fs'],['samplingfrequency'],False,False
2,autocorr,autocorr,autocorr,[],[],True,True
3,calc_centroid,calc_centroid,calc_centroid,['fs'],['samplingfrequency'],True,True
4,calc_max,calc_max,calc_max,[],[],True,True
5,calc_mean,calc_mean,calc_mean,[],[],True,True
6,calc_median,calc_median,calc_median,[],[],True,True
7,calc_min,calc_min,calc_min,[],[],True,True
8,calc_std,calc_std,calc_std,[],[],True,True
9,calc_var,calc_var,calc_var,[],[],True,True


# Estimate Speed difference

Run the same calculations as above for comparision in a timeit loop to see speed diferences

In [17]:
from timeit import Timer

In [18]:
perf = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params', 'MKR_ByName']].copy()

In [19]:
def get_time(wts, name, fn, params={}):
    try:
        fn1 = partial(fn, wts, **params)
        perf = Timer(fn1).timeit(number=numberComps) 
    except Exception as err:
        print('=== %s ===' % name)
        print(err)
        return
    return perf

def get_apply_fn(mkr_lib=True, single=True, data=wts_rand):
    wts = data[0][0] if single else data
    lookup = mkr if mkr_lib else tsfel
    lookup_params = default_mkr if mkr_lib else default_tsfel
    get_fn = lambda name: partial(ref_wrapper, getattr(lookup, name)) if (not single) and (not mkr_lib) else getattr(lookup, name)
    return lambda row: math.nan if is_nan(row[0]) else get_time(wts, row[0], get_fn(row[0]), {key: lookup_params[key] for key in row[1] if key in lookup_params})

In [20]:
%%time
if checkPerf:
    perf['MKR_single'] = overview[['MKR', 'MKR_params']].swifter.apply(get_apply_fn(mkr_lib=True, single=True), axis=1)
    perf['TSFEL_single'] = overview[['TSFEL', 'TSFEL_params']].swifter.apply(get_apply_fn(mkr_lib=False, single=True), axis=1)


CPU times: user 14 s, sys: 7.89 s, total: 21.8 s
Wall time: 10.9 s


In [21]:
%%time
if checkPerf:
    perf['MKR_multi'] = overview[['MKR', 'MKR_params']].swifter.apply(get_apply_fn(mkr_lib=True, single=False), axis=1)
    perf['TSFEL_multi'] = overview[['TSFEL', 'TSFEL_params']].swifter.apply(get_apply_fn(mkr_lib=False, single=False), axis=1)

/app/libraries/src/features/statistical.py:149: RuntimeWarning: divide by zero encountered in log2
  entr = -np.sum(p_without_null * np.log2(p_without_null), axis=-1) / normTerm



CPU times: user 5min, sys: 3min 29s, total: 8min 29s
Wall time: 3min 47s


In [22]:
if checkPerf:
    # compare values
    perf['single MKR/TSFEL'] = perf['MKR_single'] / perf['TSFEL_single'] 
    perf['multi MKR/TSFEL'] = perf['MKR_multi'] / perf['TSFEL_multi'] 
    perf['MKR Single/Multi'] = perf['MKR_single'] / perf['MKR_multi'] 
    perf['TSFEL Single/Multi'] = perf['TSFEL_single'] / perf['TSFEL_multi'] 

In [23]:
perf_related = ['MKR_single', 'TSFEL_single', 'MKR_multi', 'TSFEL_multi', 'single MKR/TSFEL', 'multi MKR/TSFEL', 'MKR Single/Multi', 'TSFEL Single/Multi']

\<library\>_single: time per call on a single-channel sequence  
\<library\>_multi: time per call on a multi-channel sequence  
single MKR/Tsfel: porportion of time of adapted and vectorized feature to reference implementation (on a single-channel sequence)  
multi MKR/Tsfel: porportion of time of adapted and vectorized feature to reference implementation (on a multi-channel sequence)  

In [24]:
perf[['TSFEL', 'MKR'] + perf_related].sort_values(by=['multi MKR/TSFEL']).style.format({'single MKR/TSFEL': cnum, 'multi MKR/TSFEL': cnum})

,TSFEL,MKR,MKR_single,TSFEL_single,MKR_multi,TSFEL_multi,single MKR/TSFEL,multi MKR/TSFEL,MKR Single/Multi,TSFEL Single/Multi
10,distance,distance,0.009330,0.088267,0.013539,2.548091,0.10570124407111092,0.005313194122816361,0.689138,0.034640
37,slope,slope,0.115335,0.118706,0.076690,2.287828,0.9715987397464498,0.033521007698079704,1.503901,0.051886
16,fft_mean_coeff,fft_mean_coeff,0.102099,0.102102,0.114248,3.143883,0.9999696382097968,0.036339771263236426,0.893660,0.032476
7,calc_min,calc_min,0.005350,0.004698,0.004014,0.106049,1.1386216584023723,0.03785154905481147,1.332727,0.044304
36,skewness,skewness,0.122870,0.122444,0.140994,3.709880,1.0034791358027741,0.03800510965293373,0.871454,0.033005
8,calc_std,calc_std,0.013392,0.013529,0.016205,0.423484,0.9898295563388184,0.03826611754015288,0.826394,0.031948
4,calc_max,calc_max,0.002855,0.002787,0.004125,0.107393,1.0245092761480166,0.03841139892528439,0.692104,0.025949
21,kurtosis,kurtosis,0.091842,0.089336,0.107494,2.790654,1.0280548223699992,0.038519357827958206,0.854391,0.032012
5,calc_mean,calc_mean,0.004405,0.005934,0.006470,0.165611,0.7422942752364861,0.0390669437823121,0.680799,0.035830
24,max_power_spectrum,max_power_spectrum,0.142169,0.152776,0.166054,4.095941,0.9305729102916592,0.04054099220725729,0.856166,0.037299


Summary: 

In most cases tsfel is as fast or faster in single-channel, while slower in multi-channel feature calculation.  
To be precise on average the reference takes 1.3 times as long as the adapted version on single channel and 0.08 times as long on multi-channel (with 15 channels)  
Which makes sense, as there sometimes is an overhead to transforming the input in order for it to be usable by numpys vectorized functions.

In [25]:
perf.describe()

,MKR_single,TSFEL_single,MKR_multi,TSFEL_multi,single MKR/TSFEL,multi MKR/TSFEL,MKR Single/Multi,TSFEL Single/Multi
count,47.000000,57.000000,47.000000,57.000000,47.000000,47.000000,47.000000,57.000000
mean,0.076880,0.121668,0.477757,3.555533,1.364296,0.093606,0.712351,0.035478
std,0.135920,0.220072,2.708196,6.502479,0.722984,0.142913,0.213540,0.006339
min,0.002855,0.002787,0.004014,0.059978,0.105701,0.005313,0.047569,0.025545
25%,0.010904,0.012219,0.016914,0.354227,1.001724,0.045544,0.653712,0.032360
50%,0.052551,0.044725,0.075409,1.098922,1.123545,0.058773,0.697389,0.033517
75%,0.080816,0.118706,0.102338,3.143883,1.589540,0.079521,0.760157,0.036815
max,0.886619,1.229068,18.638431,36.534504,4.809425,0.907118,1.503901,0.062884


In [26]:
# uncomment if you want to know perf of combined implementations 
perf['MKR_ByName multi'] = perf['MKR_multi'].fillna(perf['TSFEL_multi']).to_list()
perf_related = ['MKR_ByName multi', 'MKR_single', 'TSFEL_single', 'MKR_multi', 'TSFEL_multi', 'single MKR/TSFEL', 'multi MKR/TSFEL', 'MKR Single/Multi', 'TSFEL Single/Multi']
perf[['MKR_ByName'] + perf_related].sort_values(by=['MKR_ByName multi']).style.format({'single MKR/TSFEL': cnum, 'multi MKR/TSFEL': cnum})

,MKR_ByName,MKR_ByName multi,MKR_single,TSFEL_single,MKR_multi,TSFEL_multi,single MKR/TSFEL,multi MKR/TSFEL,MKR Single/Multi,TSFEL Single/Multi
7,calc_min,0.004014,0.005350,0.004698,0.004014,0.106049,1.1386216584023723,0.03785154905481147,1.332727,0.044304
4,calc_max,0.004125,0.002855,0.002787,0.004125,0.107393,1.0245092761480166,0.03841139892528439,0.692104,0.025949
5,calc_mean,0.006470,0.004405,0.005934,0.006470,0.165611,0.7422942752364861,0.0390669437823121,0.680799,0.035830
0,abs_energy,0.008273,0.004760,0.005197,0.008273,0.126480,0.9157479557956038,0.06540791792341333,0.575331,0.041093
35,rms,0.008848,0.006715,0.009243,0.008848,0.195942,0.726458369416285,0.04515826111945616,0.758872,0.047173
49,sum_abs_diff,0.010614,0.007385,0.006838,0.010614,0.225066,1.0799906410317892,0.04716089568633617,0.695788,0.030384
33,pk_pk_distance,0.011957,0.010239,0.012219,0.011957,0.260340,0.83795462729575,0.045930374813712184,0.856266,0.046934
27,mean_diff,0.013156,0.009625,0.009032,0.013156,0.267824,1.0656672461004555,0.049120299478911306,0.731628,0.033723
10,distance,0.013539,0.009330,0.088267,0.013539,2.548091,0.10570124407111092,0.005313194122816361,0.689138,0.034640
26,mean_abs_diff,0.014435,0.008569,0.009228,0.014435,0.284733,0.9285458860588061,0.05069528392508038,0.593636,0.032410


In [27]:
threshold = 5
perf[perf['MKR_ByName multi'] < threshold].sort_values(by=['MKR_ByName multi'])['MKR_ByName'].to_list()

['calc_min',
 'calc_max',
 'calc_mean',
 'abs_energy',
 'rms',
 'sum_abs_diff',
 'pk_pk_distance',
 'mean_diff',
 'distance',
 'mean_abs_diff',
 'zero_cross',
 'calc_std',
 'calc_var',
 'ecdf',
 'auc',
 'mean_abs_deviation',
 'calc_centroid',
 'total_energy',
 'calc_median',
 'median_diff',
 'median_abs_diff',
 'autocorr',
 'ecdf_percentile',
 'ecdf_percentile_count',
 'slope',
 'median_frequency',
 'spectral_centroid',
 'max_frequency',
 'ecdf_slope',
 'spectral_roll_on',
 'spectral_roll_off',
 'human_range_energy',
 'spectral_decrease',
 'spectral_variation',
 'spectral_distance',
 'median_abs_deviation',
 'kurtosis',
 'spectral_slope',
 'fft_mean_coeff',
 'skewness',
 'interq_range',
 'max_power_spectrum',
 'spectral_spread',
 'spectral_kurtosis',
 'spectral_entropy',
 'spectral_skewness',
 'tsfel:hist',
 'tsfel:fundamental_frequency',
 'tsfel:power_bandwidth']

# Real World Comparision

Second check to make sure the implementation is correct, load a part of the knee bandage data and check values

In [28]:
%%time
import data.csl18

dataset = data.csl18.Dataset()

seg_files, target, sessions = dataset.files_to_segments(*dataset.get_files(debug=True))

data = [dataset.load_segment(*f) for f in seg_files]

==== part00 ====
==== part01 ====
==== part02 ====
CPU times: user 1.26 s, sys: 574 ms, total: 1.83 s
Wall time: 5.66 s


In [29]:
from transformers.window import Sliding

In [30]:
wts_real = list(Sliding(length=50, overlap=0, kind='rectangular').transform((data, []))[0])
# take some 20 windows from time sequence (no need to do this on all 95 of those windows)
# linear transform to space from 0-1 as the np.assert_almost_equal only looks at the decimals
wts_real = wts_real[0][20:40,:,:] / 2**16.

In [31]:
# shape now: (nr windows, nr channels, nr samples)
wts_real.shape

(20, 19, 50)

In [32]:
if checkRealWorld:
    overview['single_channel'] = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params']].apply(partial(compare_values, wts_real[0][0]), axis=1)

=== auc ===

Arrays are not almost equal to 7 decimals

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 13.01948862
Max relative difference: 0.99800026
 x: array(0.0260878)
 y: array(13.0455764)


In [33]:
if checkRealWorld:
    overview['multi_channel'] = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params']].apply(partial(compare_values, wts_real), axis=1)

=== auc ===

Arrays are not almost equal to 7 decimals

Mismatched elements: 380 / 380 (100%)
Max absolute difference: 19.63359117
Max relative difference: 0.99810062
 x: array([[2.6087807e-02, 2.4491364e-02, 2.4494316e-02, 2.4718842e-02,
        2.4512558e-02, 2.4670410e-02, 2.8988922e-02, 2.7716064e-02,
        2.3497467e-02, 2.3863098e-02, 2.4953430e-02, 2.9599510e-02,...
 y: array([[13.0455764, 12.2591174, 12.2715648, 12.3875736, 12.2774033,
        12.3578958, 14.5158511, 13.8867838, 11.7728858, 11.95727  ,
        12.4898209, 14.8359675,  8.2301727,  9.934133 ,  5.7205433,...


/app/libraries/src/features/statistical.py:175: RuntimeWarning: divide by zero encountered in true_divide
  vals = (p_end - p_init) / (percentiles[...,1] - percentiles[...,0])
/usr/local/lib/python3.6/dist-packages/tsfel/feature_extraction/features.py:685: RuntimeWarning: divide by zero encountered in double_scalars
  return (p_end - p_init) / (x_end - x_init)
/app/libraries/src/features/statistical.py:149: RuntimeWarning: divide by zero encountered in log2
  entr = -np.sum(p_without_null * np.log2(p_without_null), axis=-1) / normTerm


In [34]:
overview.sort_values(by=['MKR']).style.format({'single_channel': cstr, 'multi_channel': cstr, 'real_single_channel': cstr, 'real_multi_channel': cstr})

,TSFEL,MKR,MKR_ByName,TSFEL_params,MKR_params,single_channel,multi_channel
0,abs_energy,abs_energy,abs_energy,[],[],True,True
1,auc,auc,auc,['fs'],['samplingfrequency'],False,False
2,autocorr,autocorr,autocorr,[],[],True,True
3,calc_centroid,calc_centroid,calc_centroid,['fs'],['samplingfrequency'],True,True
4,calc_max,calc_max,calc_max,[],[],True,True
5,calc_mean,calc_mean,calc_mean,[],[],True,True
6,calc_median,calc_median,calc_median,[],[],True,True
7,calc_min,calc_min,calc_min,[],[],True,True
8,calc_std,calc_std,calc_std,[],[],True,True
9,calc_var,calc_var,calc_var,[],[],True,True


In [35]:
perf = overview[['MKR', 'TSFEL', 'MKR_params', 'TSFEL_params', 'MKR_ByName']].copy()

In [36]:
%%time
if checkPerf:
    perf['MKR_single'] = overview[['MKR', 'MKR_params']].swifter.apply(get_apply_fn(mkr_lib=True, single=True, data=wts_real), axis=1)
    perf['TSFEL_single'] = overview[['TSFEL', 'TSFEL_params']].swifter.apply(get_apply_fn(mkr_lib=False, single=True, data=wts_real), axis=1)


CPU times: user 13.7 s, sys: 9.03 s, total: 22.7 s
Wall time: 11 s


In [37]:
%%time
if checkPerf:
    perf['MKR_multi'] = overview[['MKR', 'MKR_params']].swifter.apply(get_apply_fn(mkr_lib=True, single=False, data=wts_real), axis=1)
    perf['TSFEL_multi'] = overview[['TSFEL', 'TSFEL_params']].swifter.apply(get_apply_fn(mkr_lib=False, single=False, data=wts_real), axis=1)


CPU times: user 1h 4min 18s, sys: 43min 45s, total: 1h 48min 3s
Wall time: 49min 8s


In [38]:
if checkPerf:
    # compare values
    perf['single MKR/TSFEL'] = perf['MKR_single'] / perf['TSFEL_single'] 
    perf['multi MKR/TSFEL'] = perf['MKR_multi'] / perf['TSFEL_multi'] 
    perf['MKR Single/Multi'] = perf['MKR_single'] / perf['MKR_multi'] 
    perf['TSFEL Single/Multi'] = perf['TSFEL_single'] / perf['TSFEL_multi'] 

In [39]:
perf_related = ['MKR_single', 'TSFEL_single', 'MKR_multi', 'TSFEL_multi', 'single MKR/TSFEL', 'multi MKR/TSFEL', 'MKR Single/Multi', 'TSFEL Single/Multi']

In [40]:
perf[['TSFEL', 'MKR'] + perf_related].sort_values(by=['multi MKR/TSFEL']).style.format({'single MKR/TSFEL': cnum, 'multi MKR/TSFEL': cnum})

,TSFEL,MKR,MKR_single,TSFEL_single,MKR_multi,TSFEL_multi,single MKR/TSFEL,multi MKR/TSFEL,MKR Single/Multi,TSFEL Single/Multi
10,distance,distance,0.010121,0.086907,0.047041,32.650812,0.11646397803602307,0.001440714556935527,0.215166,0.002662
36,skewness,skewness,0.128572,0.124913,0.237003,51.354149,1.0292931880574432,0.004615072112733329,0.542491,0.002432
21,kurtosis,kurtosis,0.095639,0.092157,0.184162,37.141329,1.0377823038726612,0.00495840633467889,0.519319,0.002481
16,fft_mean_coeff,fft_mean_coeff,0.104271,0.101703,0.237613,39.220945,1.0252509510499852,0.006058321659662292,0.438828,0.002593
20,interq_range,interq_range,0.126673,0.119740,0.313873,48.247474,1.0578971104041643,0.006505472182849178,0.403580,0.002482
24,max_power_spectrum,max_power_spectrum,0.142389,0.149006,0.369907,53.188388,0.9555931825736654,0.006954653348304017,0.384933,0.002801
5,calc_mean,calc_mean,0.004413,0.004666,0.015000,1.873367,0.945881646912932,0.008006921015158822,0.294215,0.002491
35,rms,rms,0.006850,0.008325,0.021496,2.578741,0.8228807552853734,0.008335926652595997,0.318680,0.003228
8,calc_std,calc_std,0.013732,0.013883,0.046457,5.296833,0.9891376379383543,0.00877077007814154,0.295583,0.002621
9,calc_var,calc_var,0.011441,0.011135,0.040914,4.622700,1.0274532343004479,0.008850692934836783,0.279635,0.002409


In [41]:
perf.describe()

,MKR_single,TSFEL_single,MKR_multi,TSFEL_multi,single MKR/TSFEL,multi MKR/TSFEL,MKR Single/Multi,TSFEL Single/Multi
count,47.000000,57.000000,47.000000,57.000000,47.000000,47.000000,47.000000,57.000000
mean,0.076935,0.123176,5.226698,46.892080,1.397677,0.047199,0.257556,0.002798
std,0.134825,0.222884,34.069207,84.957786,0.773737,0.160563,0.108415,0.000550
min,0.002984,0.002757,0.015000,0.583050,0.116464,0.001441,0.003762,0.002006
25%,0.011122,0.011846,0.046786,4.622700,0.999531,0.009731,0.196565,0.002491
50%,0.053137,0.042927,0.214930,14.765302,1.113745,0.016133,0.246976,0.002621
75%,0.077760,0.115149,0.311082,39.220945,1.612323,0.021883,0.307589,0.002855
max,0.879665,1.200270,233.813877,467.511904,4.826504,1.013009,0.542491,0.004866
